Computing the true position of the Sun
======================================

In [ ]:
# Imports

from kanon.calendars import Calendar, Date
from kanon.tables import HTable
from kanon.units import Sexagesimal
from kanon.units.precision import set_precision, TruncatureMode
import astropy.units as u

In [ ]:
# 3rd of July 1327

year = 1327
month = 7
day = 3

In [ ]:
# Day computation

# We're using the Julian A.D. calendar

calendar = Calendar.registry["Julian A.D."]
date = Date(calendar, (year, month, day))

# We need to express the numbers of days from the start of the calendar
# in Sexagesimal representation.

days = Sexagesimal.from_int(int(date.days_from_epoch()))

days

In [ ]:
# Defining a function resolving a position from a mean motion table

def position_from_table(tab, radix, zodiac_offset=4):
    # Starting with the radix

    result = radix * u.degree

    # Adding days
    for i, v in enumerate(days[:]):
        result += tab.get(v) >> i + zodiac_offset

    # Conversion in degrees modulo 6 zodiacal signs

    result %= Sexagesimal(6, 0) * u.degree
    return result

In [ ]:
# Sun mean position

# Reading the table from DISHAS
tab_mean_motion = HTable.read(193)
tab_mean_motion

In [ ]:
# Mean position from days, table, and radix

mean_sun_pos = position_from_table(tab_mean_motion, Sexagesimal("4,38;21,0,30,28"))
mean_sun_pos

In [ ]:
# Fixed stars

tab_fixed_stars = HTable.read(236)

mean_fixed_star_pos = position_from_table(tab_fixed_stars, Sexagesimal(0))
mean_fixed_star_pos

In [ ]:
# Access and recess position

tab_access_recess = HTable.read(237)

access_recess_pos = position_from_table(
    tab_access_recess, Sexagesimal("5,59;12,34"), zodiac_offset=2
)
access_recess_pos

In [ ]:
# Access and recess equation

tab_eq_access_recess = HTable.read(238)

with set_precision(tmode=TruncatureMode.ROUND):
    eq_access_recess = tab_eq_access_recess.get(access_recess_pos)
eq_access_recess

In [ ]:
# Sun apogee

with set_precision(tmode=TruncatureMode.ROUND):
    solar_apogee_pos = (
        mean_fixed_star_pos + eq_access_recess + Sexagesimal("1,11;25,23") * u.degree
    )
solar_apogee_pos

In [ ]:
# Sun mean argument

with set_precision(tmode=TruncatureMode.ROUND):
    mean_arg_sun = mean_sun_pos - solar_apogee_pos
    if mean_arg_sun < 0:
        mean_arg_sun += 360
mean_arg_sun

In [ ]:
# Sun equation

tab_eq_sun = HTable.read(19)
with set_precision(tmode=TruncatureMode.ROUND):
    eq_sun = tab_eq_sun.get(mean_arg_sun)
eq_sun

In [ ]:
# Sun true position

with set_precision(pmode=2, tmode=TruncatureMode.ROUND):
    true_pos_sun = mean_sun_pos - eq_sun
    if true_pos_sun < 0:
        true_pos_sun += 360
true_pos_sun